### 1.1 Preprocessing - Reinhard Normalization and WSI Tiling

As a first preprocessing step, all slides were color normalized with respect to a reference image selected by an expert neuropathologist. Color normalization was performed using the method described by [Reinhard et. al](https://ieeexplore.ieee.org/document/946629).

The resulting color normalized whole slide images were tiled using PyVips to generate 1536 x 1536 images patches.

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pyvips as Vips
from tqdm import tqdm

import vips_utils, normalize

In [2]:
RAW_DIR = '/home/kangway/data/cnn_path/raw_images/plaques/'
SAVE_DIR = '/home/kangway/data/cnn_path/tiled/03_21_18/'

In [3]:
ref_imagename = 'NA4749-02_AB.svs'

In [4]:
imagenames = sorted(os.listdir(RAW_DIR))

In [5]:
imagenames.remove('NA5005-02_AB-002.svs')
imagenames.append('NA5005-02_AB-002.svs')

In [6]:
%%time
# Load reference image, fit Reinhard normalizer
ref_image = Vips.Image.new_from_file(RAW_DIR + ref_imagename, level=0)

normalizer = normalize.Reinhard()
normalizer.fit(ref_image)

CPU times: user 42min 45s, sys: 1min 22s, total: 44min 8s
Wall time: 7min 11s


In [7]:
for imagename in tqdm(imagenames[:-1]):
    vips_img = Vips.Image.new_from_file(RAW_DIR + imagename, level=0)
    out = normalizer.transform(vips_img)
    out.filename = vips_img.filename
    vips_utils.save_and_tile(out, SAVE_DIR)

100%|██████████| 32/32 [4:22:44<00:00, 492.65s/it]  


In [9]:
# Resize the single 40x image down to 20x
for imagename in tqdm(imagenames[-1:]):
    vips_img = Vips.Image.new_from_file(RAW_DIR + imagename, level=0)
    vips_img = vips_img.resize(0.5)
    out = normalizer.transform(vips_img)
    out.filename = vips_img.filename
    vips_utils.save_and_tile(out, SAVE_DIR)

100%|██████████| 1/1 [34:37<00:00, 2077.97s/it]


In [ ]:
# Takes 42 minutes to apply and tile a 40x image.